In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA


In [2]:
subsidies = pd.read_csv("../data_clean/subsidies.csv")


In [3]:
subsidies.head(13)

,subsidy_year,Care,Education,education_yoy,care_yoy,t,edu_lag1,edu_lag2,care_lag1,care_lag2,edu_roll3,care_roll3
0,2015,NaN,3.263588e+06,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1.503780e+05,8.999615e+07,26.575833,NaN,1,3.263588e+06,NaN,NaN,NaN,NaN,NaN
2,2017,1.341066e+07,5.668109e+07,-0.370183,88.179641,2,8.999615e+07,3.263588e+06,1.503780e+05,NaN,4.998028e+07,NaN
3,2018,3.953753e+07,7.778580e+07,0.372341,1.948217,3,5.668109e+07,8.999615e+07,1.341066e+07,1.503780e+05,7.482101e+07,1.769952e+07
4,2019,4.881995e+07,1.007047e+08,0.294641,0.234775,4,7.778580e+07,5.668109e+07,3.953753e+07,1.341066e+07,7.839053e+07,3.392271e+07
5,2020,4.253882e+07,1.078756e+08,0.071207,-0.128659,5,1.007047e+08,7.778580e+07,4.881995e+07,3.953753e+07,9.545535e+07,4.363210e+07
6,2021,1.508918e+08,7.653167e+07,-0.290556,2.547156,6,1.078756e+08,1.007047e+08,4.253882e+07,4.881995e+07,9.503731e+07,8.075019e+07
7,2022,1.633902e+08,8.412070e+07,0.099162,0.082830,7,7.653167e+07,1.078756e+08,1.508918e+08,4.253882e+07,8.950931e+07,1.189403e+08
8,2023,2.521113e+08,8.343322e+07,-0.008173,0.543001,8,8.412070e+07,7.653167e+07,1.633902e+08,1.508918e+08,8.136186e+07,1.887978e+08
9,2024,1.955329e+08,8.572724e+07,0.027495,-0.224419,9,8.343322e+07,8.412070e+07,2.521113e+08,1.633902e+08,8.442705e+07,2.036781e+08


In [4]:
subsidies_clean = subsidies[["subsidy_year","Education","Care"]].copy()

# remove future year accidentally included
subsidies_clean = subsidies_clean[subsidies_clean["subsidy_year"] <= 2025]

# remove NaN rows
subsidies_clean = subsidies_clean.dropna()

# set index
subsidies_clean = subsidies_clean.set_index("subsidy_year")

subsidies_clean


,Education,Care
subsidy_year,,
2016,8.999615e+07,1.503780e+05
2017,5.668109e+07,1.341066e+07
2018,7.778580e+07,3.953753e+07
2019,1.007047e+08,4.881995e+07
2020,1.078756e+08,4.253882e+07
2021,7.653167e+07,1.508918e+08
2022,8.412070e+07,1.633902e+08
2023,8.343322e+07,2.521113e+08
2024,8.572724e+07,1.955329e+08


In [8]:
sub = subsidies_clean.copy()



In [9]:
sub.head()

,Education,Care
subsidy_year,,
2016,8.999615e+07,150378.00
2017,5.668109e+07,13410656.00
2018,7.778580e+07,39537529.87
2019,1.007047e+08,48819946.79
2020,1.078756e+08,42538817.81


In [10]:
train = sub.loc[:2022]
test  = sub.loc[2023:]


In [11]:
def evaluate(y_true, y_pred):

    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return mae, rmse, mape


In [12]:
def linear_forecast(train, test, column):

    X_train = np.arange(len(train)).reshape(-1,1)
    y_train = train[column].values

    model = LinearRegression().fit(X_train, y_train)

    X_test = np.arange(len(train), len(train)+len(test)).reshape(-1,1)
    pred = model.predict(X_test)

    return pred


In [13]:
lr_pred_edu  = linear_forecast(train, test, "Education")
lr_pred_care = linear_forecast(train, test, "Care")

print("LR Education:", evaluate(test["Education"], lr_pred_edu))
print("LR Care:", evaluate(test["Care"], lr_pred_care))


LR Education: (9109582.240357155, np.float64(9137869.895988794), np.float64(10.712906684649031))
LR Care: (30137772.859880943, np.float64(44744754.60908429), np.float64(12.281487498624992))


In [14]:
def holt_forecast(train, test, column):

    model = ExponentialSmoothing(
        train[column],
        trend="add",
        seasonal=None
    ).fit()

    pred = model.forecast(len(test))
    return pred.values


In [16]:
holt_pred_edu  = holt_forecast(train, test, "Education")
holt_pred_care = holt_forecast(train, test, "Care")

print("Holt Education:", evaluate(test["Education"], holt_pred_edu))
print("Holt Care:", evaluate(test["Care"], holt_pred_care))


Holt Education: (7134609.587424055, np.float64(7840648.491464126), np.float64(8.344637260835041))
Holt Care: (43373285.39246773, np.float64(52017822.92886604), np.float64(17.9573401416401))


In [17]:
def arima_forecast(train, test, column):

    model = ARIMA(train[column], order=(1,1,1)).fit()
    pred = model.forecast(len(test))
    return pred.values


In [18]:
arima_pred_edu  = arima_forecast(train, test, "Education")
arima_pred_care = arima_forecast(train, test, "Care")

print("ARIMA Education:", evaluate(test["Education"], arima_pred_edu))
print("ARIMA Care:", evaluate(test["Care"], arima_pred_care))


c:\Users\HOTGNVXSQ\Desktop\FORMATION_AI_INSTITUTE\PROJECT_AMSTERDAM\env\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\HOTGNVXSQ\Desktop\FORMATION_AI_INSTITUTE\PROJECT_AMSTERDAM\env\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\HOTGNVXSQ\Desktop\FORMATION_AI_INSTITUTE\PROJECT_AMSTERDAM\env\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


ARIMA Education: (2517861.4316701093, np.float64(2680568.322922808), np.float64(2.947655740759069))
ARIMA Care: (27706509.12298605, np.float64(38740459.86693185), np.float64(11.568693671328385))


In [19]:
def compare_models(column):

    results = {}

    results["LR"]   = evaluate(test[column], linear_forecast(train,test,column))
    results["HOLT"] = evaluate(test[column], holt_forecast(train,test,column))
    results["ARIMA"]= evaluate(test[column], arima_forecast(train,test,column))

    return pd.DataFrame(results, index=["MAE","RMSE","MAPE"]).T.sort_values("MAPE")


In [21]:
print("Education model ranking")
print(compare_models("Education"))

print("\nCare model ranking")
print(compare_models("Care"))


Education model ranking


c:\Users\HOTGNVXSQ\Desktop\FORMATION_AI_INSTITUTE\PROJECT_AMSTERDAM\env\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                MAE          RMSE       MAPE
ARIMA  2.517861e+06  2.680568e+06   2.947656
HOLT   7.134610e+06  7.840648e+06   8.344637
LR     9.109582e+06  9.137870e+06  10.712907

Care model ranking


c:\Users\HOTGNVXSQ\Desktop\FORMATION_AI_INSTITUTE\PROJECT_AMSTERDAM\env\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\HOTGNVXSQ\Desktop\FORMATION_AI_INSTITUTE\PROJECT_AMSTERDAM\env\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                MAE          RMSE       MAPE
ARIMA  2.770651e+07  3.874046e+07  11.568694
LR     3.013777e+07  4.474475e+07  12.281487
HOLT   4.337329e+07  5.201782e+07  17.957340


In [25]:
future_years = [2026, 2027, 2028]

edu_model  = ARIMA(sub["Education"], order=(1,1,1)).fit()
care_model = ARIMA(sub["Care"], order=(1,1,1)).fit()

edu_forecast  = edu_model.forecast(3)
care_forecast = care_model.forecast(3)

forecast_subsidy = pd.DataFrame({
    "subsidy_year": future_years,
    "Education": edu_forecast.values,
    "Care": care_forecast.values
})

forecast_subsidy


c:\Users\HOTGNVXSQ\Desktop\FORMATION_AI_INSTITUTE\PROJECT_AMSTERDAM\env\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


,subsidy_year,Education,Care
0,2026,8.580058e+07,2.039021e+08
1,2027,8.582612e+07,2.328175e+08
2,2028,8.581936e+07,2.071334e+08


In [26]:
forecast_subsidy_final = pd.concat([sub.reset_index(), forecast_subsidy], ignore_index=True)    

In [27]:
forecast_subsidy_final.head()

,subsidy_year,Education,Care
0,2016,8.999615e+07,150378.00
1,2017,5.668109e+07,13410656.00
2,2018,7.778580e+07,39537529.87
3,2019,1.007047e+08,48819946.79
4,2020,1.078756e+08,42538817.81


In [28]:
forecast_subsidy_final.to_csv("../TRANSFORMATION/forecast_subsidy_final.csv", index=False)